In [ ]:
# default_cls_lvl 3

# Setting up an UPboard
> A guide for setting up and [Upboard Intel SBC](https://up-board.org/upsquared/specifications/)


### Ubuntu 18.04  
>The upboard come with Ubuntu 16.04. We decided to upgrade to 18.04


Install ubuntu 18.04 from usb, on boot press F7 to set the boot sequence in necessary.  
Set up a user `rov` with a password  

__Edit grub file__     - to set USB memory and file system check and repair  

`sudo nano /etc/default/grub`  

Find and replace  

```
GRUB_CMDLINE_LINUX_DEFAULT="quiet splash usbcore.usbfs_memory_mb=1000"
GRUB_CMDLINE_LINUX="reboot=efi fsck.mode=force fsck.repair=yes"
GRUB_TIMEOUT=5
```

Update the grub file  

```
sudo update-grub
sudo reboot
```

To test that buffer_mb = 1000  
`cat /sys/module/usbcore/parameters/usbfs_memory_mb`  
Check fsck boot log  
`journalctl -u systemd-fsck*`  

### Install SSH 
> To allow remote terminal access  


`sudo apt install openssh-server`  
Connect remotely with ethernet cable and find the upboard  address with  
`arp -a`  
Connect to the upboard rov user with the found address i.e `10.42.0.114`  
`ssh rov@10.42.0.114`  
If you get "Host key verification failed"
`ssh-keygen -R hostname`             i.e ssh-keygen -R 10.42.0.114  

### Install SAMBA
> optional but allows easy file copy to and from the rov via pc   
https://magpi.raspberrypi.org/articles/samba-file-server


```
sudo apt-get update
sudo apt-get upgrade
sudo apt-get install samba samba-common-bin```
Edit the smb config file  
`sudo nano /etc/samba/smb.conf`
```
[sambashare]
Comment = Rov shared folder
Path = /home/rov
Browseable = yes
Writeable = Yes
only guest = no
create mask = 0777
directory mask = 0777
Public = yes
Guest ok = yes
```
Password & restart  
`sudo smbpasswd -a rov`  
`sudo /etc/init.d/samba restart`  
From the pc with file navigator or explorer connect to `smb://10.42.0.xxx/sambashare`
  

### Install Ubuntu kernel 4.15.0 7 & mraa
> for UP from PPA on Ubuntu 18.04  - to enable GPIO 
> (see https://wiki.up-community.org/Ubuntu)  


```
sudo add-apt-repository ppa:ubilinux/up
sudo apt update
sudo apt-get autoremove --purge 'linux-.*generic'   (type no to abort removal)
sudo apt-get install linux-image-generic-hwe-18.04-upboard
sudo apt dist-upgrade -y
sudo reboot
```
__Install mraa__  - to enable GPIO  (see https://wiki.up-community.org/MRAA/UPM)
```
sudo add-apt-repository ppa:mraa/mraa
sudo apt-get update
sudo apt-get install mraa-tools mraa-examples libmraa1 libmraa-dev libupm-dev libupm1 upm-examples
sudo apt-get install python-mraa python3-mraa node-mraa libmraa-java
```

### Install virtualenv and CV2
> optional to keep python dependencies separate  

see https://itnext.io/virtualenv-with-virtualenvwrapper-on-ubuntu-18-04-goran-aviani-d7b712d906d5  


```
sudo apt install python3-pip
pip3 install virtualenv virtualenvwrapper
nano ~/.bashrc
```
add these to the end of the file  
```
#Virtualenvwrapper settings:
export VIRTUALENVWRAPPER_PYTHON=/usr/bin/python3
export WORKON_HOME=$HOME/.virtualenvs
export VIRTUALENVWRAPPER_VIRTUALENV=/home/rov/.local/bin/virtualenv
source ~/.local/bin/virtualenvwrapper.sh
```
Run  
`source ~/.bashrc`  
 
Make a virtualenv for the camera server  

```
mkvirtualenv flir
Workon flir
```
__Install CV2__ 
```
sudo apt install python3-opencv
pip install opencv-contrib-python
```
To test 
``` 
python
import cv2
```

### Install FLIR camera Spinaker & python sdk
> FLIR camera drivers  


Download spinaker amd64 from
`https://flir.app.boxcn.net/v/SpinnakerSDK/folder/74729115388`  
Extract to a director such as  `home/rov/spinnaker-1.27.0.48-amd64`  
Follow instructions in readme i.e
```
cd ~/spinnaker-1.27.0.48-amd64
sudo sh install_spinnaker.sh        # add rov as a user
```
Download Python sdk  
https://flir.app.boxcn.net/v/SpinnakerSDK/folder/74728781416  
```
workon flir
cd ~/git/FLIR-pubsub/run
python -m pip install spinnaker_python-1.27.0.48-cp36-cp36m-linux_x86_64.whl
```
You may need to reboot to allow usb driver to work on user rov  


### Install the FLIR server from github
> The the server polls connected cameras and runs them depending on client demand.  
It requires a yaml file for each camera connected. 


```
sudo apt install git
git clone https://github.com/johnnewto/FLIR_pubsub.git
```
__To run server__
```
workon flir 
cd ~/git/FLIR-pubsub/run
python flir-server.py
```

__To install as a service__  
https://www.raspberrypi.org/documentation/linux/usage/systemd.md
```
sudo cp flir-server.service /etc/systemd/system/flir-server.service
sudo systemctl start flir-server.service
sudo systemctl stop flir-server.service
sudo systemctl enable flir-server.service   # enable at boot
```
To show status of the service   
`sudo systemctl status flir-server.service`  
To show fsck log  
`journalctl -u systemd-fsck*`

You will need to set up the camera yaml files  
see [Camera-yaml-files](https://johnnewto.github.io/FLIR_pubsub/FLIR_server_utils/#Camera-yaml-files)  